In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

from vayu_gnn.dbx.dbx_config import dbx_helper, DropboxHelper

In [2]:
patna = dbx_helper.read_csv(dbx_helper.raw_input_path, 'node_locations', 'Patna sensor locations.csv')
guru = dbx_helper.read_csv(dbx_helper.raw_input_path, 'node_locations', 'Gurugram sensor locations.csv')

In [3]:
patna

,S. No.,Sensor ID,Sensor Tyoe,Geo co-ordinates,Location
0,1,TARA012,Static,"25.599409, 85.113762",Ecopark
1,2,TARA013,Static,"25.606958, 85.165016",Rajendra Nagar
2,3,TARA016,Static,"25.578127, 85.056877","Phulwari Shrif, Hadaspura"
3,4,TARA017,Static,"25.622450, 85.091530",Ramnagri More Ashiana Road
4,5,TARA018,Static,"25.582579, 85.097511","Shivpuri, Anisabad"
5,6,TARA019,Static,"25.569492, 85.179970",Near IGIMS Bio Waste Management Plant
6,7,TARA021,Static,"25.610352, 85.132950",NaN
7,8,TARA022,Static,"25.619673, 85.147415",SK Memorial Hall
8,9,TARA025,Static,"25.614792, 85.099220",North Shastri Nagar
9,10,TARA026,Static,"25.636068, 85.104805","Kurji, Patliputra Industrial Area (BSPCB)"


In [26]:
# Drop if missing Geo co-ordinates
patna = patna.dropna(subset=['Geo co-ordinates'])
guru = guru.dropna(subset=['Geo co-ordinates'])

# Separate the Geo co-ordinates column lat and long on the comma. Both should be numeric
patna[['lat', 'long']] = patna['Geo co-ordinates'].str.split(', ', expand=True).apply(pd.to_numeric)

guru['Geo co-ordinates'] = guru['Geo co-ordinates'].str.replace('O', '0')
guru['Geo co-ordinates'] = guru['Geo co-ordinates'].str.replace('770053172', '77.0053172')

guru[['lat', 'long']] = guru['Geo co-ordinates'].str.split(', ', expand=True).apply(pd.to_numeric)

# Create two dictionaries, the keys are the Sensor ID and the values are dictionaries with lat and long
patna_dict = patna.set_index('Sensor ID')[['lat', 'long']].T.to_dict()
guru_dict = guru.set_index('Sensor ID')[['lat', 'long']].T.to_dict()

In [27]:
dbx_helper.write_pickle(patna_dict, dbx_helper.clean_input_path, 'node_locations/sensors_only', f'Patna_static.pickle')
dbx_helper.write_pickle(guru_dict, dbx_helper.clean_input_path, 'node_locations/sensors_only', f'Gurugram_static.pickle')

File 'Patna_static.pickle' successfully uploaded to Dropbox path: '/input/clean/node_locations/sensors_only/Patna_static.pickle'
File 'Gurugram_static.pickle' successfully uploaded to Dropbox path: '/input/clean/node_locations/sensors_only/Gurugram_static.pickle'


In [ ]:
cities = ['Patna', 'Gurugram']

for city in cities:
    devices = dbx_helper.read_pickle(dbx_helper.clean_input_path, 'node_locations/sensors_only', f'{city}_static.pickle')

    # Calculate the center (average latitude and longitude) from the existing devices
    center_lat = float(sum(device['lat'] for device in devices.values()) / len(devices))
    center_long = float(sum(device['long'] for device in devices.values()) / len(devices))

    half_side = 1
    offsets = np.linspace(-half_side, half_side, 9)  # 9 equally spaced points from -1 to 1

    extra_nodes = {}
    node_counter = 1

    # Top side: fixed latitude (center_lat + 1), longitude varies by offset
    for offset in offsets:
        device_id = f"extra_node_{node_counter}"
        lat_val = float(round(center_lat + half_side, 4))
        long_val = float(round(center_long + offset, 4))
        extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
        node_counter += 1

    # Bottom side: fixed latitude (center_lat - 1), longitude varies by offset
    for offset in offsets:
        device_id = f"extra_node_{node_counter}"
        lat_val = float(round(center_lat - half_side, 4))
        long_val = float(round(center_long + offset, 4))
        extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
        node_counter += 1

    # Left side: fixed longitude (center_long - 1), latitude varies by offset (excluding corners)
    for offset in offsets[1:-1]:
        device_id = f"extra_node_{node_counter}"
        lat_val = float(round(center_lat + offset, 4))
        long_val = float(round(center_long - half_side, 4))
        extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
        node_counter += 1

    # Right side: fixed longitude (center_long + 1), latitude varies by offset (excluding corners)
    for offset in offsets[1:-1]:
        device_id = f"extra_node_{node_counter}"
        lat_val = float(round(center_lat + offset, 4))
        long_val = float(round(center_long + half_side, 4))
        extra_nodes[device_id] = {'lat': lat_val, 'long': long_val}
        node_counter += 1

    combined_devices = {**devices, **extra_nodes}

    dbx_helper.write_pickle(combined_devices, dbx_helper.clean_input_path, f'node_locations/{city}', f'nodes.pickle')

    # Also save as a gdf
    records = []
    for device_id, coords in combined_devices.items():
        records.append({
            'device_id': device_id,
            'lat': coords['lat'],
            'long': coords['long']
        })

    df = pd.DataFrame(records)
    df['geometry'] = df.apply(lambda row: Point(row['long'], row['lat']), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

    dbx_helper.write_shp(gdf, dbx_helper.clean_input_path, f'node_locations/{city}/gdf', f'nodes')

File 'nodes.pickle' successfully uploaded to Dropbox path: '/input/clean/node_locations/Gurugram/nodes.pickle'
Shapefile written to temporary directory: /var/folders/w2/spn01gpx39d_btp2kmjt17jh0000gn/T/tmp4rekxw7q
File 'nodes.shp' successfully uploaded to Dropbox at '/input/clean/node_locations/Gurugram/gdf/nodes.shp'
File 'nodes.shx' successfully uploaded to Dropbox at '/input/clean/node_locations/Gurugram/gdf/nodes.shx'
File 'nodes.dbf' successfully uploaded to Dropbox at '/input/clean/node_locations/Gurugram/gdf/nodes.dbf'
File 'nodes.prj' successfully uploaded to Dropbox at '/input/clean/node_locations/Gurugram/gdf/nodes.prj'
File 'nodes.cpg' successfully uploaded to Dropbox at '/input/clean/node_locations/Gurugram/gdf/nodes.cpg'
